In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import ee
import multiprocessing
import geemap
import geetools
import datetime

In [2]:
ee.Initialize(project='ee-jonstar')

## CONUS Geometry

In [3]:
"""
Points of interest for each urban area.
KEY:
First and second points = Landsat
If the second point is zero, then there is only one Landsat image needed to cover that city and its surrounding area
"""
city_points = {
    'DMV':[ee.Geometry.Point(-76.6122, 39.2904), 0],
    'NYC':[ee.Geometry.Point(-73.274, 40.688), 0],
    'Atlanta':[ee.Geometry.Point(-84.485, 33.641), ee.Geometry.Point(-84.039, 34.155)],
    'Miami':[ee.Geometry.Point(-80.267, 25.818), 0],
    'Chicago':[ee.Geometry.Point(-88.037, 41.826), 0],
    'Phoenix':[ee.Geometry.Point(-112.02, 33.46), ee.Geometry.Point(-111.896, 34.199)],
    'Denver':[ee.Geometry.Point(-104.47, 39.99), ee.Geometry.Point(-104.72, 39.44)],
    'Seattle':[ee.Geometry.Point(-121.952, 47.546), 0],
    'San_Francisco':[ee.Geometry.Point(-122.179, 37.745), 0],
    'Los_Angeles':[ee.Geometry.Point(-118.365, 34.279), ee.Geometry.Point(-118.744, 33.383)],
    'Toronto':[ee.Geometry.Point(-80.273, 43.289), ee.Geometry.Point(-79.609, 44.187)],
    'Mexico_City':[ee.Geometry.Point(-99.101, 19.012), ee.Geometry.Point(-99.187, 20.084)],
    'Las_Vegas':[ee.Geometry.Point(-115.162, 36.154), 0],
    'Salt_Lake_City':[ee.Geometry.Point(-111.896, 40.752), 0],
    'Dallas':[ee.Geometry.Point(-96.858, 32.789), 0],
    'Houston':[ee.Geometry.Point(-94.816, 30.034), ee.Geometry.Point(-95.356, 29.399)],
    'New_Orleans':[ee.Geometry.Point(-90.102, 29.947), 0],
    'St_Louis':[ee.Geometry.Point(-90.505, 38.644), 0],
    'Minneapolis':[ee.Geometry.Point(-93.248, 44.969), ee.Geometry.Point(-92.998, 45.555)],
    'Jacksonville':[ee.Geometry.Point(-81.357, 30.292), 0],
    'Charlotte':[ee.Geometry.Point(-80.797, 35.028), ee.Geometry.Point(-80.573, 35.63)],
    'Philadelphia':[ee.Geometry.Point(-75.133, 39.938), 0],
    'San_Diego':[ee.Geometry.Point(-117.088, 32.647), 0],
    'San_Juan':[ee.Geometry.Point(-66.311, 18.421), 0],
    'Montreal':[ee.Geometry.Point(-73.082, 45.629), ee.Geometry.Point(-73.353, 45.148)],
    'Guadalajara':[ee.Geometry.Point(-103.35, 20.663), 0],
    'Monterrey':[ee.Geometry.Point(-100.17, 25.897), 0],
    'Cancun':[ee.Geometry.Point(-86.592, 21.402), ee.Geometry.Point(-86.725, 20.709)],
    'Billings':[ee.Geometry.Point(-108.5, 45.78), 0],
    'Guatemala_City':[ee.Geometry.Point(-90.527, 14.623), 0],
    'San_Jose':[ee.Geometry.Point(-84.082, 9.937), 0],
    'Havana':[ee.Geometry.Point(-82.36, 23.128), 0],
    'Santo_Domingo':[ee.Geometry.Point(-69.822, 18.681), 0],
    'Tegucigalpa':[ee.Geometry.Point(-87.206, 14.171), 0],
    'Managua':[ee.Geometry.Point(-86.269, 12.017), ee.Geometry.Point(-86.145, 12.349)],
    'Panama_City':[ee.Geometry.Point(-79.522, 8.98), 0],
    'Bogota':[ee.Geometry.Point(-74.074, 4.697), 0],
    'Lima':[ee.Geometry.Point(-76.887, -11.835), ee.Geometry.Point(-76.99, -12.383)],
    'Quito':[ee.Geometry.Point(-78.507, -.232), 0],
    'Santiago':[ee.Geometry.Point(-70.667, -33.46), 0],
    'Buenos_Aires':[ee.Geometry.Point(-58.543, -34.621), 0],
    'Sao_Paulo':[ee.Geometry.Point(-46.644, -23.564), 0],
    'Manaus':[ee.Geometry.Point(-60.037, -3.089), 0],
    'Punta_Arenas':[ee.Geometry.Point(-70.179, -52.885), ee.Geometry.Point(-71.881, -52.806)],
    'La_Paz':[ee.Geometry.Point(-67.965, -16.125), ee.Geometry.Point(-67.885, -17.004)],
    'Montevideo':[ee.Geometry.Point(-56.175, -34.902), 0],
    'Brasilia':[ee.Geometry.Point(-47.890, -15.799), 0],
    'Caracas':[ee.Geometry.Point(-66.898, 10.475), 0]
}

In [189]:
"""
Export coordinates for each urban area
KEY: [utm zone, boolean T/F for northern/southern hemisphere, utm x, utm y]
"""
export_coords = {
    'DMV':[18, True, 292000, 4372200],
    'NYC':[18, True, 562773, 4535140],
    'Phoenix':[12, True, 356592, 3758141],
    'Miami':[17, True, 503991, 2915332],
    'Chicago':[16, True, 386195, 4680677],
    'Denver':[13, True, 466349, 4442816],
    'Seattle':[10, True, 518589, 5311884],
    'San_Francisco':[10, True, 541523, 4206919],
    'Los_Angeles':[11, True, 346524, 3803779],
    'Atlanta':[16, True, 703143, 3787639],
    'Toronto':[17, True, 566749, 4867878],
    'Mexico_City':[14, True, 426632, 2177328],
    'Las_Vegas':[11, True, 618776, 4038562],
    'Salt_Lake_City':[12, True, 371135, 4538777],
    'Dallas':[14, True, 639754, 3678675],
    'Houston':[15, True, 231750, 3334911],
    'New_Orleans':[15, True, 723933, 3361201],
    'St_Louis':[15, True, 672050, 4323318],
    'Minneapolis':[15, True, 433510, 5031071],
    'Jacksonville':[17, True, 401561, 3376745],
    'Charlotte':[17, True, 472912, 3938294],
    'Philadelphia':[18, True, 445926, 4469797],
    'San_Diego':[11, True, 468102, 3666860],
    'San_Juan':[19, True, 732184, 2071833],
    'Montreal':[18, True, 564088, 5089380],
    'Guadalajara':[13, True, 621576, 2326245],
    'Monterrey':[14, True, 318956, 2885247],
    'Cancun':[16, True, 470776, 2369100],
    'Billings':[12, True, 655301, 5109344],
    'Guatemala_City':[15, True, 711929, 1659823],
    'San_Jose':[16, True, 772982, 1148166],
    'Havana':[17, True, 310663, 2566532],
    'Santo_Domingo':[19, True, 354586, 2108039],
    'Tegucigalpa':[16, True, 405549, 1616590],
    'Managua':[16, True, 524984, 1388932],
    'Panama_City':[17, True, 610881, 1043658],
    'Bogota':[18, True, 546442, 561743],
    'Lima':[18, False, 262348, 8709133],
    'Quito':[17, True, 733395, 16923],
    'Santiago':[19, False, 298834, 6329569],
    'Buenos_Aires':[21, False, 326030, 6204959],
    'Sao_Paulo':[23, False, 294104, 7446208],
    'Manaus':[20, False, 762422, 9716468],
    'Punta_Arenas':[19, False, 319805, 4162998],
    'La_Paz':[19, False, 548109, 8213756],
    'Montevideo':[21, False, 545458, 6223320],
    'Brasilia':[22, False, 783269, 8287283],
    'Caracas':[19, True, 687497, 1176029]
}

In [247]:
# Pull points to make data for a specific city (look above for options)
city = 'Phoenix'
points = city_points[city]
city_export = export_coords[city]
if city_export[1]:
    crs_prefix = '326'
else:
    crs_prefix = '327'
utm_proj = f'EPSG:{crs_prefix}{city_export[0]}'
point = ee.Geometry.Point([city_export[2]+30*2999/2,city_export[3]-30*2999/2], utm_proj)
export_region = point.buffer(distance=44990, proj=utm_proj).bounds(proj=utm_proj)
city_export

[12, True, 356592, 3758141]

## Landsat Initialization

In [248]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [249]:
landsat = landsat.filterDate('2022-01-01', '2024-01-01')
landsat_US = landsat.filterBounds(points[0])
if points[1] != 0:
    #pt1_filt = ee.Filter.bounds(points[0])
    #pt2_filt = ee.Filter.bounds(points[1])
    #Landsat_pts_filt = ee.Filter.Or(pt1_filt, pt2_filt)
    #landsat_US = landsat.filter(Landsat_pts_filt)
    landsat_US2 = landsat.filterBounds(points[1])
    print('Combine Landsat')
elif points[0] == 0:
    raise Exception('Points not formatted correctly')

Combine Landsat


In [250]:
landsat_US.first().projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32612',
 'transform': [30, 0, 242085, 0, -30, 3789915]}

In [251]:
# Preprocessing function
def process_multiple_Landsat(image):
    ######################################
    # Combines Landsat images matched by time into one image
    
    match = ee.Image(image.get('matched_img'))
    mosaic_image = ee.ImageCollection([match, image]).mosaic() # "image" goes second because .mosaic() puts the last image on top

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(image.get('system:time_start'))       
        return f.set('system:time_start', Landsat_time)

    # Add timestamp back in
    return_image = get_Landsat_time(mosaic_image)
    
    return return_image

In [252]:
# Join sequential Landsat images
if points[1] != 0:
    if city == 'Punta_Arenas':
        landsat_US = landsat_US.filterBounds(points[1])
    else:
        best_join = ee.Join.saveBest(matchKey='matched_img', measureKey='time_diff', outer=False)
        # Difference value is equal to 1 minute (for sequential Landsat images)
        timeFiltLandsat = ee.Filter.maxDifference(difference=60e3, leftField='system:time_start', rightField='system:time_start')
        Landsat_near = ee.ImageCollection(best_join.apply(landsat_US, landsat_US2, timeFiltLandsat))
        landsat_US = Landsat_near.map(process_multiple_Landsat)
        print('Landsat join')

Landsat join


In [253]:
print(landsat_US.size().getInfo())

45


## Sentinel-1 SAR Initialization

In [254]:
sentinel = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate('2022-01-01', '2024-01-01').filterBounds(export_region)

In [255]:
slist = sentinel.toList(10)

In [256]:
ee.Image(slist.get(1)).select('angle').projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32612',
 'transform': [-12267.305123955826,
  -3884.9141599542927,
  414279.54206890916,
  2468.157742620446,
  -20024.44737518346,
  3924785.5848947177]}

In [257]:
print(sentinel.size().getInfo())

252


## Filtering images by time

In [258]:
# Join Landsat and Sentinel images
all_join_LS = ee.Join.saveAll(matchesKey='matched_img', measureKey='time_diff', outer=True)
# Difference value is equal to 6 days (Full Sentinel-1 orbital period) in milliseconds
timeFiltLandsatSentinel = ee.Filter.maxDifference(difference=5184e5, leftField='system:time_start', rightField='system:time_start')
LandsatSentinel_near = ee.ImageCollection(all_join_LS.apply(landsat_US, sentinel, timeFiltLandsatSentinel))

In [259]:
LandsatSentinel_near.size().getInfo()

45

## Helper Functions

In [122]:
# Applies scale and offset factors for Landsat bands
def scale_and_offset_Landsat(image):
    reflect_weight = 0.0000275
    reflect_bias = -0.2
    reflect_image = image.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(reflect_weight).add(reflect_bias)

    LST_weight = 0.00341802
    LST_bias = 149
    LST_image = image.select('ST_B10').multiply(LST_weight).add(LST_bias)

    return reflect_image.addBands(srcImg=LST_image, names=['ST_B10'])

In [19]:
# Function to add missing bands
def add_missing_bands(image):
    # Define the complete set of bands you want to work with
    all_bands = ['VV', 'VH', 'HH', 'HV', 'angle']
    constants = ee.Dictionary({'VV':2, 'VH':2, 'HH':2, 'HV':2, 'angle':-999})
    
    # Get the bands present in the image
    existing_bands = image.bandNames()
    # Determine the missing bands
    missing_bands = ee.List(all_bands).removeAll(existing_bands)
    # Add missing bands with a default value of 2
    filled_image = missing_bands.iterate(
        lambda band, img: ee.Image(img).addBands(ee.Image.random().add(ee.Image.constant(constants.get(band))).rename([ee.String(band)])),
        image
    )
    # Ensure the image has all bands in the correct order
    return ee.Image(filled_image).select(all_bands)

def process_multiple_sentinel(feature):
    Sentinel_col = ee.ImageCollection(ee.List(feature.get('matched_img')))
    
    # Define the complete set of bands you want to work with
    all_bands = ['VV', 'VH', 'HH', 'HV', 'angle']

    # Want most recent images last in collection for mosaic
    sorted_collection = Sentinel_col.sort('time_diff', ascending=False).map(add_missing_bands)
    Sentinel_image = sorted_collection.mosaic()
    
    return Sentinel_image.cast({'angle':'double'})

In [20]:
# Preprocessing function
def process_Landsat_Sentinel(feature):
    ######################################
    # Sentinel portion
               
    # Three-case if:
    # 1st: More than one image match -> combine into one image
    # Second: No image match -> return blank image
    # Third: One image match -> return singular image
    sentinel_image = ee.Algorithms.If(ee.List(feature.get('matched_img')).length().gt(ee.Number(1)),
                     process_multiple_sentinel(feature),
                     ee.Algorithms.If(ee.List(feature.get('matched_img')).length().lt(ee.Number(1)),
                                      add_missing_bands(ee.Image()),
                                      add_missing_bands(ee.Image(ee.List(feature.get('matched_img')).get(0)).cast({'angle':'double'})))
                                     )
    
    #####################################
    # Landsat portion
    
    # Scaling and offset
    feature = ee.Image(feature)
    #image1 = ee.Image(feature.get('primary'))
    landsat = scale_and_offset_Landsat(feature)
    #landsat_image = scale_and_offset_Landsat(feature)
    # Add cloud mask back in
    landsat_image = landsat.addBands(srcImg=feature, names=['QA_PIXEL'])

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(feature.get('system:time_start'))       
        return f.set('timestamp', Landsat_time)

    # Add timestamp back in
    landsat_image = get_Landsat_time(landsat_image)

    ##########################################
    image = landsat_image.addBands(srcImg=sentinel_image).cast({'QA_PIXEL':'double'})
    #image = landsat_image.cast({'QA_PIXEL':'double'})
    
    return image

## Produce Landsat/Sentinel geotiffs and their respective times

In [21]:
def times_to_features(num):
    return ee.Feature(None, {'value': num})

In [260]:
processed2 = LandsatSentinel_near.map(process_Landsat_Sentinel)

In [163]:
plist = processed2.toList(30)

In [302]:
ee.Image(plist.get(0)).getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7651, 7781],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 527085, 0, -30, 4582215]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7651, 7781],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 527085, 0, -30, 4582215]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7651, 7781],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 527085, 0, -30, 4582215]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7651, 7781],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 527085, 0, -30, 4582215]},
  {'id': 'SR_B6',
   'data_type': {'type'

In [261]:
processed2.size().getInfo()

45

In [27]:
times2 = processed2.aggregate_array('timestamp')
features2 = ee.FeatureCollection(times2.map(times_to_features))

In [28]:
task = ee.batch.Export.table.toDrive(
    collection=features2,
    description=f'Landsat_times_{city}',
    #folder=f'Landsat_times',
    fileFormat='CSV',
)
task.start()

## Export stuff

In [262]:
# Must first download the times files created above in the previous two sections
ls_times = pd.read_csv(f'/Users/jonstar/Documents/heat_data/Landsat_times/Landsat_times_{city}.csv').value

In [263]:
datetime.datetime.fromtimestamp(ls_times[3]/1000, datetime.UTC).strftime('%Y%m%d%H%M')

'202202191804'

In [264]:
len(ls_times)

45

In [119]:
# Export Landsat/Sentinel images
num = processed2.size().getInfo()
#num = 4
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    time_str = toTimezone(datetime.fromtimestamp(ls_times[i]/1000), pytz.utc).strftime('%Y%m%d%H%M')
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{time_str}', fileFormat='GeoTIFF', folder='Landsat_SAR_DMV_new',
                dimensions='3000x3000', crs=f'EPSG:{crs_prefix}{city_export[0]}', crsTransform=[30, 0, city_export[1], 0, -30, city_export[2]],
        formatOptions={'cloudOptimized':True})
    task.start()

In [114]:
# Export Landsat images for export to GEE asset for visualizing
num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toAsset(
        img, description=f'Landsat_Sentinel_image_{i}', assetId=f'projects/ee-jonstar/assets/{city}_LS_test',
                dimensions='3000x3000', crs=f'EPSG:{crs_prefix}{city_export[0]}', crsTransform=[30, 0, city_export[1], 0, -30, city_export[2]])
    task.start()

In [265]:
# Export Landsat/Sentinel images new way
num = processed2.size().getInfo()
#num = 1
img_list = processed2.toList(num)
point = ee.Geometry.Point([city_export[2]+30*2999/2,city_export[3]-30*2999/2], f'EPSG:{crs_prefix}{city_export[0]}')

for i in list(range(num)):
#for i in list(range(15,16)):
    img = ee.Image(img_list.get(i))
    time_str = datetime.datetime.fromtimestamp(ls_times[i]/1000, datetime.UTC).strftime('%Y%m%d%H%M')
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{city}_{time_str}', fileFormat='GeoTIFF', #folder=f'Landsat_SAR_{city}',
        region = export_region, crs=utm_proj, scale=30,
        formatOptions={'cloudOptimized':True})
    task.start()

In [208]:
# Export Landsat/Sentinel images new way as assets
#num = processed2.size().getInfo()
num = 1
img_list = processed2.toList(num)
point = ee.Geometry.Point([city_export[2]+30*2999/2,city_export[3]-30*2999/2], f'EPSG:{crs_prefix}{city_export[0]}')

for i in list(range(num)):
#if i in [19]:
    img = ee.Image(img_list.get(i))
    time_str = datetime.datetime.fromtimestamp(ls_times[i]/1000, datetime.UTC).strftime('%Y%m%d%H%M')
    
    task = ee.batch.Export.image.toAsset(
        img, description=f'Landsat_Sentinel_image_{time_str}', assetId=f'projects/ee-jonstar/assets/{city}_LS_test',
        region = export_region,
        crs='EPSG:4326', scale=30)
    task.start()